In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import json

In [3]:
agents = ["AlwaysDefect","AlwaysCooperate","TitforTat","NotTitforTat"]

In [4]:
df = pd.read_csv("data.csv")

In [5]:
df.head()

,agent1,agent2,agent3,agent4,dynamics,game,gamma,generations_until_stability,initial_distribution
0,"[0.25, 0.24999861992278735]","[0.25, 0.24999964329358812]","[0.25, 0.2500007774585631]","[0.25, 0.25000095932506144]",replicator,Prisoner's Dilemma,0.95,2,"[0.25, 0.25, 0.25, 0.25]"
1,"[0.4, 0.39999918521554895]","[0.4, 0.39999456172877124]","[0.1, 0.10000060587442994]","[0.1, 0.10000095738949004]",replicator,Prisoner's Dilemma,0.95,2,"[0.4, 0.4, 0.1, 0.1]"
2,"[0.1, 0.10000161263107654]","[0.1, 0.09999998528957198]","[0.4, 0.39999669239762814]","[0.4, 0.3999969159197778]",replicator,Prisoner's Dilemma,0.95,2,"[0.1, 0.1, 0.4, 0.4]"
3,"[0.4, 0.39999381669185446]","[0.1, 0.1000012171656001]","[0.4, 0.39999356082182824]","[0.1, 0.10000193845597924]",replicator,Prisoner's Dilemma,0.95,2,"[0.4, 0.1, 0.4, 0.1]"
4,"[0.1, 0.10000365922363344]","[0.4, 0.3999935113382955]","[0.1, 0.09999847479606966]","[0.4, 0.39999795258289217]",replicator,Prisoner's Dilemma,0.95,2,"[0.1, 0.4, 0.1, 0.4]"


In [11]:
df[{'gamma': .95}]

TypeError: unhashable type: 'dict'

## Use these graphs to find which combinations are 'interesting'

In [ ]:
import seaborn

plt.rcParams["figure.figsize"] = (10,5)
table = []

def handle(df, M):
    M = np.cumsum(np.random.randn(32, 4, 1000), 2)
    M -= M.min(1, keepdims=True)
    M /= M.sum(1, keepdims=True)
    xs = range(M.shape[2])
    ys = M.mean(0)
    stds = .1 * M.std(0) #+ 0.1 * np.random.randn(M.shape[1], M.shape[2])
    cmap = plt.cm.get_cmap('hsv', 4)
    
    int_entropy = -np.sum(ys * np.log(ys), axis=0)
    
    plt.plot(int_entropy, alpha=.10, linewidth=2, label='entropy')
    plt.legend()
    plt.twinx()
    
    for i in range(4):
        plt.grid(True, 'major', 'y', ls='-', lw=.5, c='k', alpha=.3)
        plt.plot(xs, ys[i], label=agents[i], color=cmap(i), linewidth=.5)
        plt.fill_between(xs, ys[i]-2 * stds[i], ys[i]+2 * stds[i], alpha=0.10, facecolor='red', color=cmap(i))
        plt.fill_between(xs, ys[i]-stds[i], ys[i]+stds[i], alpha=0.15, facecolor='red', color=cmap(i))
    plt.title("{}, {} Dynamics, $\\gamma$: {}".format(df.iloc[0]["game"], df.iloc[0]["dynamics"].capitalize(), df.iloc[0]["gamma"]))
    plt.ylabel('Population Profile  $\\theta$')
    plt.xlabel('Time')
    
    plt.legend()
    plt.ylim((0,1))
    plt.xlim(0, xs[-1])
    plt.show()
    
    
    table.append('{} & {} & ${}$ & ${}$ & ${:.2f}$\\\\'.format(df.iloc[0]["game"], 
                                              df.iloc[0]["dynamics"].capitalize(),
                                              df.iloc[0]["gamma"],
                                              df.iloc[0]["generations_until_stability"], 
                                              np.mean(int_entropy)))

sims = []
for dynamics in df['dynamics'].unique():
    for gamma in df['gamma'].unique():
        for game in df['game'].unique():
            for distro in df['initial_distribution'].unique():
                subset = df[(df['dynamics'] == dynamics) & (df['gamma'] == gamma) & (df['game'] == game) & (df['initial_distribution'] == distro)]

                data = []
                for i, r in subset.iterrows():
                    plays1 = np.array(json.loads(df.iloc[i]['agent1']))
                    plays2 = np.array(json.loads(df.iloc[i]['agent2']))
                    plays3 = np.array(json.loads(df.iloc[i]['agent3']))
                    plays4 = np.array(json.loads(df.iloc[i]['agent4']))
                    M = np.vstack([plays1, plays2, plays3, plays4])
                    data.append(M)

                M = np.array(data)
                
                handle(subset, M)
                
len(table)

In [ ]:
for i in range(12):
    plays1 = np.array(json.loads(df.iloc[i]['agent1']))
    plays2 = np.array(json.loads(df.iloc[i]['agent2']))
    plays3 = np.array(json.loads(df.iloc[i]['agent3']))
    plays4 = np.array(json.loads(df.iloc[i]['agent4']))
    plt.plot(plays1, label=agents[0])
    plt.plot(plays2, label=agents[1])
    plt.plot(plays3, label=agents[2])
    plt.plot(plays4, label=agents[3])
    plt.title("{}\ngamma: {}".format(df.iloc[i]["game"], df.iloc[i]["gamma"]))
    plt.legend()
    plt.ylim((0,1))
    plt.show()

In [ ]:
df['agent1_cat'] = df.agent1.astype("category").cat.codes
df['agent2_cat'] = df.agent2.astype("category").cat.codes
df['game_cat'] = df['agent1_cat'] + df['agent2_cat']

df.head()

y = df['win']
X = df[['game_cat', 'n_plays']]

mod = sm.OLS(y, X)    # Describe model

res = mod.fit()       # Fit model

print(res.summary()) 